# Preprocess the raw survey data

Saves a cleaned version of the survey data to be used in analysis and that can be publicly published (is IRB compliant).

Keeps only data from participants who:

- Have valid Prolific IDs
- Completed the survey
- Passed ALL attention checks

Keeps only the data columns necessary for analysis. 
Drops:
- Prolific ID (privacy risk)
- Unneeded qualtrics metadata

Saves fields metadata to a separate file and drops this metadata from the saved cleaned data.

In [1]:
import numpy as np
import pandas as pd

raw_data_filepath = '../data/sample-raw.csv'
cleaned_data_filepath = '../data/sample-preprocessed.csv'
fields_filepath = '../data/fields.csv'

In [2]:
df = pd.read_csv(raw_data_filepath, dtype='str')
print(df.shape)

(4029, 850)


Can immediately drop the columns never used that were added by Qualtrics.

In [3]:
drop_cols = [
    'StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
    'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 
    'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 
    'UserLanguage',
    # use PROLIFIC_PID instead which is set as embedded data via a URL parameter
    'Q_PROLIFIC_PID',
]
df = df.drop(drop_cols, axis=1)

In [4]:
non_choice_fields = [f for f in df.columns if not (f.startswith('CHOICE_'))]
print(list(non_choice_fields))
df[non_choice_fields].drop([1]).drop(['PROLIFIC_PID'], axis=1).head()

['Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'Q-attn-number-21', 'Q_online_shopping', 'Q_takeout_food', 'Q_online_groceries', 'Q_habits_attn_check', 'Q_v_0_why_no', 'Q_v_0_why_no_4_TEXT', 'Q_v_1', 'Q_v_1_why_no', 'Q_v_1_why_no_4_TEXT', 'Q_v_2', 'Q_v_2_why_no', 'Q_v_2_why_no_4_TEXT', 'Q_v_attn_check_1', 'Q_gender', 'Q_age', 'Q_race', 'Q_income', 'Q_urban_rural', 'Q_residence_type', 'Q_residence_type_3_TEXT', 'Q_state', 'Q_comments', 'PROLIFIC_PID', 'STUDY_ID', 'online_shopping', 'takeout_food_delivery', 'online_groceries', 'prescription_medications_delivery', 'last_minute_groceries_delivery', 'liquor_store_delivery', 'questions_asked']


,Duration (in seconds),Finished,RecordedDate,ResponseId,Q-attn-number-21,Q_online_shopping,Q_takeout_food,Q_online_groceries,Q_habits_attn_check,Q_v_0_why_no,...,Q_state,Q_comments,STUDY_ID,online_shopping,takeout_food_delivery,online_groceries,prescription_medications_delivery,last_minute_groceries_delivery,liquor_store_delivery,questions_asked
0,Duration (in seconds),Finished,Recorded Date,Response ID,Please select the number shown.,How often (on average) do you make online shop...,How often (on average) do you have take-out fo...,How often (on average) do you purchase your gr...,How closely are you paying attention? This is ...,What are your reasons for never ordering t...,...,"50 States, D.C. and Puerto Rico",Thank you for taking the survey. When you cont...,STUDY_ID,online_shopping,takeout_food_delivery,online_groceries,prescription_medications_delivery,last_minute_groceries_delivery,liquor_store_delivery,questions_asked
2,128,1,2022-09-12 08:14:07,R_2chgn4rkY4fPpOO,21,3,3,3,5,NaN,...,NaN,NaN,62facea5645985d77801fd22,True,True,True,True,True,True,NaN
3,91,1,2022-09-12 08:16:36,R_237SFIQ3gnVTXpg,21,5,2,5,"1,2,3,4,5",NaN,...,22,NaN,62facea5645985d77801fd22,True,True,False,True,False,False,NaN
4,262,1,2022-09-12 08:25:22,R_2zUNSdeQI7uzbbu,21,4,2,5,"1,2,3,4,5",NaN,...,22,alex test -- take-out food only,62facea5645985d77801fd22,True,True,False,False,False,False,12
5,129,1,2022-09-12 08:33:47,R_3DbUHlWnnu89bWC,21,5,1,5,"1,2,3,4,5",NaN,...,22,geoffrey test - takeout only,62facea5645985d77801fd22,True,True,False,False,False,False,12


Save the fields metadata (rows 0,1) in case someone wants them for easy reference later. 

Do not save the choice Q's in the fields data.

Then drop metadata from the preprocessed data.

In [5]:
fields_df = df[non_choice_fields].loc[0].rename('field').to_frame()
fields_df

,field
Duration (in seconds),Duration (in seconds)
Finished,Finished
RecordedDate,Recorded Date
ResponseId,Response ID
Q-attn-number-21,Please select the number shown.
Q_online_shopping,How often (on average) do you make online shop...
Q_takeout_food,How often (on average) do you have take-out fo...
Q_online_groceries,How often (on average) do you purchase your gr...
Q_habits_attn_check,How closely are you paying attention? This is ...
Q_v_0_why_no,What are your reasons for never ordering t...


In [6]:
print('saving fields to file %s' % fields_filepath)
fields_df.to_csv(fields_filepath)

saving fields to file ../data/fields.csv


Drop participants' data based on exclusion criteria.

In [7]:
df = df.drop([0,1])
starting_respondent_N = df['ResponseId'].nunique()
print('N=%s total responses before data cleaning' % starting_respondent_N)
df.drop(['PROLIFIC_PID'], axis=1).head(3)

N=4027 total responses before data cleaning


,Duration (in seconds),Finished,RecordedDate,ResponseId,Q-attn-number-21,Q_online_shopping,Q_takeout_food,Q_online_groceries,Q_habits_attn_check,Q_v_0_why_no,...,Q_state,Q_comments,STUDY_ID,online_shopping,takeout_food_delivery,online_groceries,prescription_medications_delivery,last_minute_groceries_delivery,liquor_store_delivery,questions_asked
2,128,1,2022-09-12 08:14:07,R_2chgn4rkY4fPpOO,21,3,3,3,5,NaN,...,NaN,NaN,62facea5645985d77801fd22,True,True,True,True,True,True,NaN
3,91,1,2022-09-12 08:16:36,R_237SFIQ3gnVTXpg,21,5,2,5,"1,2,3,4,5",NaN,...,22,NaN,62facea5645985d77801fd22,True,True,False,True,False,False,NaN
4,262,1,2022-09-12 08:25:22,R_2zUNSdeQI7uzbbu,21,4,2,5,"1,2,3,4,5",NaN,...,22,alex test -- take-out food only,62facea5645985d77801fd22,True,True,False,False,False,False,12


In [8]:
TEST_PROLIFIC_IDS = [
    '62a2a94d2c2d3cac1cdb636b',
    '62fac4a55e4fc878ec1b0c76',
]
# only include responses completed by Prolific participants
N_PROLIFIC_PID_na = len(df[df['PROLIFIC_PID'].isna()])
print('dropping %s non PROLIFIC responses' % N_PROLIFIC_PID_na)
df = df[df['PROLIFIC_PID'].notna()]
N_uncompleted = len(df[df['Finished'] != '1'])
df = df[df['Finished'] == '1']
print('dropping %s incomplete responses' % N_uncompleted)
# Remove the test responses filled out by us
N_test = len(df[df['PROLIFIC_PID'].isin(TEST_PROLIFIC_IDS)])
print('dropping %s test responses' % N_test)
df = df[~df['PROLIFIC_PID'].isin(TEST_PROLIFIC_IDS)]
print('%s completed responses' % len(df))

dropping 0 non PROLIFIC responses
dropping 2 incomplete responses
dropping 9 test responses
4016 completed responses


In [9]:
def passed_attn_checks(row):
    try:
        if row['Q-attn-number-21'] != '21':
            print('%s : %s %s' % (row['PROLIFIC_PID'], row['ResponseId'], 'failed Q-attn-number-21'))
            return False
        if (str(row['Q_habits_attn_check']) == 'nan') or (len(row['Q_habits_attn_check'].split(',')) != 5):
            print('%s : %s %s' % (row['PROLIFIC_PID'], row['ResponseId'], 'failed Q_habits_attn_check'))
            return False
        if row['Q_v_attn_check_1'] in ['1','2']: # only selected 1 option - either 1 or 2
            print('%s : %s %s' % (row['PROLIFIC_PID'], row['ResponseId'], 'failed Q_v_attn_check_1'))
            return False
    except Exception as e:
        print('exception for %s : %s' % (row['PROLIFIC_PID'], row['ResponseId']))
        raise e
    return True

In [10]:
df['passed_attn_checks'] = df.apply(passed_attn_checks, axis=1)
passed_attn_checks_ResponseId_list = list(df[df.passed_attn_checks == True].ResponseId)
pass_rate = len(passed_attn_checks_ResponseId_list)/len(df)
print('%0.2f = %s/%s respondents passed all attention checks' % (
    pass_rate, len(passed_attn_checks_ResponseId_list), len(df)))
# passed_attn_checks_ResponseId_list

5a8599792a842c0001fcbacb : R_25QqBuJ38NHBT9s failed Q-attn-number-21
5c832bd24b0de2001b3510fc : R_1PdBN81pQZz10MZ failed Q_v_attn_check_1
5f490d10022f3f2e8f33cc9a : R_1jp41iLW1oDOfxM failed Q_v_attn_check_1
610f7619672322f2c0c98396 : R_BD6TwN3suFU9hjb failed Q_v_attn_check_1
5eee7448b465303350093b53 : R_6XslJdUp7GN3B85 failed Q_v_attn_check_1
6010f9387969064a53ea6b69 : R_3g7uNDwWjSesDES failed Q_v_attn_check_1
5786d6076475d400015a959e : R_PUGtyOIfzxv8KoF failed Q-attn-number-21
631bcfaa7faf35d3abb91cbc : R_2Vr9PYvpwZPWJYT failed Q_v_attn_check_1
5bc11bbbc11e1200018dfa3b : R_XG1xQ7XxoDMEjIJ failed Q_v_attn_check_1
60fdc45f0ae9e5d153012de9 : R_2f3vUv8pwovPJuS failed Q_v_attn_check_1
6313bca352b1b9dacd2446e5 : R_1da7BFtzV6chf9W failed Q_v_attn_check_1
60ddff2cd6cb7a81cdfb6a00 : R_2t3szyR8tQ7NYJ3 failed Q_v_attn_check_1
5e7b815adb5aba05af6e6cc7 : R_wN48C2doNom9QOZ failed Q_v_attn_check_1
5e833e342598d4053db08f1f : R_25ZSN2jFhmRF7gq failed Q_habits_attn_check
61531681e3a3743f9ce59135 : R_1c

In [11]:
print('dropping responses from participants who failed any attention check')
df = df[df['passed_attn_checks']]
N = len(df)
print('N=%s' % N)

dropping responses from participants who failed any attention check
N=3715


Can now drop columns 'Finished' and 'PROLIFIC_PID' and columns corresponding to attention checks.

In [12]:
df = df.drop([
    'PROLIFIC_PID','Finished',
    'Q-attn-number-21',
    'Q_habits_attn_check',
    'Q_v_attn_check_1',
    'passed_attn_checks',
], axis=1)
df.head(3)

,Duration (in seconds),RecordedDate,ResponseId,Q_online_shopping,Q_takeout_food,Q_online_groceries,Q_v_0_why_no,Q_v_0_why_no_4_TEXT,Q_v_1,Q_v_1_why_no,...,Q_state,Q_comments,STUDY_ID,online_shopping,takeout_food_delivery,online_groceries,prescription_medications_delivery,last_minute_groceries_delivery,liquor_store_delivery,questions_asked
6,215,2022-09-12 09:25:01,R_3dNdOyEHwO3X5Tg,2,1,4,NaN,NaN,1,NaN,...,31,NaN,631f4adf9c7bc459e1650eb6,True,True,True,True,True,True,NaN
7,176,2022-09-12 09:29:21,R_1r7Zvnp6IL6dFXo,3,4,5,NaN,NaN,1,NaN,...,5,NaN,631f4adf9c7bc459e1650eb7,True,True,False,True,False,True,NaN
8,358,2022-09-12 09:29:38,R_1JIXl9IZdC0IxJt,2,4,4,NaN,NaN,3,NaN,...,11,NaN,631f4adf9c7bc459e1650eb7,True,True,True,True,True,True,NaN


How long did the survey take people on average?

(in minutes)
Use Median (50%) because there were outliers that took a while.

In [13]:
(df['Duration (in seconds)'].astype(int).describe()/60).rename('minutes')

count    61.916667
mean      5.570000
std       3.789526
min       1.050000
25%       3.500000
50%       4.616667
75%       6.333333
max      77.933333
Name: minutes, dtype: float64

Save cleaned data to file.

In [14]:
print('saving data (N=%s) to %s...' % (len(df),cleaned_data_filepath))
df.to_csv(cleaned_data_filepath, index=False)
print('...saved')

saving data (N=3715) to ../data/sample-preprocessed.csv...
...saved


In [15]:
# reading from file
clean_df = pd.read_csv(cleaned_data_filepath)
print('N=%s responses' % len(clean_df))
clean_df.head(3)

N=3715 responses


,Duration (in seconds),RecordedDate,ResponseId,Q_online_shopping,Q_takeout_food,Q_online_groceries,Q_v_0_why_no,Q_v_0_why_no_4_TEXT,Q_v_1,Q_v_1_why_no,...,Q_state,Q_comments,STUDY_ID,online_shopping,takeout_food_delivery,online_groceries,prescription_medications_delivery,last_minute_groceries_delivery,liquor_store_delivery,questions_asked
0,215,2022-09-12 09:25:01,R_3dNdOyEHwO3X5Tg,2,1,4,NaN,NaN,1,NaN,...,31,NaN,631f4adf9c7bc459e1650eb6,True,True,True,True,True,True,NaN
1,176,2022-09-12 09:29:21,R_1r7Zvnp6IL6dFXo,3,4,5,NaN,NaN,1,NaN,...,5,NaN,631f4adf9c7bc459e1650eb7,True,True,False,True,False,True,NaN
2,358,2022-09-12 09:29:38,R_1JIXl9IZdC0IxJt,2,4,4,NaN,NaN,3,NaN,...,11,NaN,631f4adf9c7bc459e1650eb7,True,True,True,True,True,True,NaN
